In [1]:
import pandas as pd
import nibabel as nib
from os.path import join, exists

df = pd.read_csv('PET_AUTOPSY.csv')

In [2]:
for indx_, row_ in df.iterrows():
    
    ad_pathology = row_['ALZHEIMER PATHOLOGY']
    lb_pathology = row_['LEWY PATHOLOGY']
    
    if ad_pathology > 1 and lb_pathology <= 1:
        
        df.loc[indx_,'GRUPO'] = 'pure-AD'
    
    elif ad_pathology > 1 and lb_pathology > 1:
        
        df.loc[indx_,'GRUPO'] = 'Mixed'
    
    elif ad_pathology <= 1 and lb_pathology > 1:
        
        df.loc[indx_,'GRUPO'] = 'pure-LB'
    
    else:
        
        df.loc[indx_,'GRUPO'] = 'Negative'

df = df.loc[df['GRUPO'] != 'Negative']
df = df.loc[df['GRUPO'] != 'Mixed']

df
        

In [15]:
import numpy as np

dir_patients = '/Users/jsilva/data/MIA/Autopsia'

atlas_img = nib.load('cortex_Harvard_Oxford_orig_atlas.nii.gz')
atlas_data = atlas_img.get_fdata()
atlas_csv = 'Harvard_Oxford.csv'
atlas_df = pd.read_csv(atlas_csv)


for indx_, row_ in df.iterrows():
    
    subject_id = row_['SUBJECT_ID']
    img_id = row_['FDG_ID']
    pet_name = join(dir_patients, f'{subject_id}_I{img_id}', 'swfdg_normhist.hdr')
    #print(pet_name)
    if exists(pet_name):
        img = nib.load(pet_name)
        data = img.get_fdata()
        
        for indx_atlas, row_atlas in atlas_df.iterrows():
            roi_name = row_atlas['ROI_NAME']
            roi_n = row_atlas['ROI_NUM']
            where_roi = np.where(atlas_data == roi_n)
            
            mean_roi = data[where_roi].mean()
            df.loc[indx_,roi_name] = mean_roi
            
df        

In [4]:
df_ad = df.loc[df['GRUPO'] == 'pure-AD']
df_lewy = df.loc[df['GRUPO'] == 'pure-LB']


In [5]:
df_lewy

In [6]:
from scipy.stats import ttest_ind

for indx_, row_ in atlas_df.iterrows():
    
    roi_name = row_['ROI_NAME']
    t_val, p_val = ttest_ind(df_ad[roi_name], df_lewy[roi_name], axis=0, equal_var=True)
    
    if p_val < 0.005:
        print(f"{roi_name}: {p_val:.3f}")
    
    else:
        df.drop(roi_name, axis=1, inplace=True)
        
df    

In [7]:
X = df.iloc[:,-3:]

y = df['GRUPO']

X

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

model = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight="balanced", random_state=None, solver='lbfgs', max_iter=100, verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

model.fit(X,y)

In [9]:
y_pred = model.predict(X)
conf_matrix = confusion_matrix(y, y_pred)
accuracy_score = accuracy_score(y, y_pred)
print(accuracy_score)
conf_matrix

In [10]:
from sklearn.svm import SVC

model = SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight="balanced", verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)

model.fit(X,y)


In [11]:
y_pred = model.predict(X)
conf_matrix = confusion_matrix(y, y_pred)
#accuracy_score = accuracy_score(y, y_pred)
#print(accuracy_score)
conf_matrix


In [13]:
df.to_csv('training_features.csv', index=False)

In [14]:
import joblib

joblib.dump(model, 'my_model.pkl')